mainly looking at https://github.com/ngessert/isic2019/blob/master/models.py

# start

In [22]:
PRFX = 'MelPrp0626A2'

# setup

In [36]:
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
import random
import os, sys
import datetime
from collections import defaultdict, Counter
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import GroupKFold

def dtnow(): return datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

In [24]:
p_cmp = '../input/siim-isic-melanoma-classification'
p_19 = '../input/andrewmvd--isic-2019'
p_out=f'../output/{PRFX}'; Path(p_out).mkdir(exist_ok=True,parents=True)

train,test,sample_submission = (pd.read_csv(o) for o in [f'{p_cmp}/{o}.csv' 
                                                         for o in ('train', 'test', 'sample_submission')])
print([o.shape for o in (train,test,sample_submission)])

train19 = pd.read_csv(f'{p_19}/ISIC_2019_Training_GroundTruth.csv')
train19meta = pd.read_csv(f'{p_19}/ISIC_2019_Training_Metadata.csv')
print(train19.shape)

display(train19.head(2))
display(train.head(2))

test.head(2)

[(33126, 8), (10982, 5), (10982, 2)]
(25331, 10)


,image,MEL,NV,BCC,AK,BKL,DF,VASC,SCC,UNK
0,ISIC_0000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ISIC_0000001,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
0,ISIC_2637011,IP_7279968,male,45.0,head/neck,unknown,benign,0
1,ISIC_0015719,IP_3075186,female,45.0,upper extremity,unknown,benign,0


,image_name,patient_id,sex,age_approx,anatom_site_general_challenge
0,ISIC_0052060,IP_3579794,male,70.0,NaN
1,ISIC_0052349,IP_7782715,male,40.0,lower extremity


# prep

## merge dfs

In [28]:
assert np.all(train19.image==train19meta.image)
train19['patient_id']=train19meta['lesion_id']

train['source'] = '20'
train['im_pth'] = f'{p_cmp}/jpeg/train/'+train.image_name+'.jpg'
train19['source'] = '19'
train19['target'] = train19.MEL
train19['im_pth'] = f'{p_19}/ISIC_2019_Training_Input/ISIC_2019_Training_Input/'+train19.image+'.jpg'


cols2use=['source', 'im_pth', 'patient_id', 'target', ]
train_all = pd.concat([train[cols2use], train19[cols2use]])
display(train_all.shape)
display(train_all.source.value_counts())
display(train_all.groupby('source').target.mean())

(58457, 4)

20    33126
19    25331
Name: source, dtype: int64

source
19    0.178516
20    0.017630
Name: target, dtype: float64

In [29]:
train_all.head()

,source,im_pth,patient_id,target
0,20,../input/siim-isic-melanoma-classification/jpe...,IP_7279968,0.0
1,20,../input/siim-isic-melanoma-classification/jpe...,IP_3075186,0.0
2,20,../input/siim-isic-melanoma-classification/jpe...,IP_2842074,0.0
3,20,../input/siim-isic-melanoma-classification/jpe...,IP_6890425,0.0
4,20,../input/siim-isic-melanoma-classification/jpe...,IP_8723313,0.0


In [34]:
train_all.patient_id.value_counts(dropna=False)

NaN            2084
IP_7279968      115
IP_4938382      115
IP_4382720      115
IP_4479736      115
               ... 
HAM_0006190       1
HAM_0003217       1
HAM_0005963       1
HAM_0000452       1
HAM_0005938       1
Name: patient_id, Length: 13904, dtype: int64

In [33]:
assert set(train.patient_id)&set(train19.patient_id)==set()

In [37]:
train_all.to_csv(f'{p_out}/train_all.csv', index=False)